In [145]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import xgboost as xgb
from sklearn import svm
from Utils import GeoUtils

from scipy import stats
from scipy.stats import norm, skew #for some statistics

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.multioutput import MultiOutputRegressor
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC
from sklearn.kernel_ridge import KernelRidge
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from matplotlib.pyplot import figure
from sklearn.decomposition import PCA

%matplotlib inline

In [146]:
def getDistance(real, predict):
    lat1 = real[:,0]
    lon1 = real[:,1]
    lat2 = predict[:,0]
    lon2 = predict[:,1]

    return np.array(GeoUtils.GeoUtils.distanceInKm(lat1, lon1,lat2, lon2))

In [147]:
# data = pd.read_csv("Dados/LocTreino_Equipe_4.csv")
data = pd.read_csv("Dados/LocTest.csv")

#Retire the nan
data['pathBTS1'] = data['pathBTS1'].fillna(data['pathBTS1'].mean())
data['taBTS2'] = data['taBTS2'].fillna(data['taBTS2'].mean())

#Retire the absurds values pathBTS1 , pathBTS4, pathBTS6
a1 = np.array(data['pathBTS1'].values.tolist())
data['pathBTS1'] = np.where(a1 > 200, data['pathBTS1'].mean(), a1).tolist()

a4 = np.array(data['pathBTS4'].values.tolist())
data['pathBTS4'] = np.where(a4 > 200, data['pathBTS4'].mean(), a4).tolist()

a6 = np.array(data['pathBTS6'].values.tolist())
data['pathBTS6'] = np.where(a6 > 200, data['pathBTS6'].mean(), a6).tolist()



bts_data = pd.read_csv("Dados/dados_BTSs.csv")

In [148]:
data.describe()

,pontoId,lat,lon,pathBTS1,pathBTS2,pathBTS3,pathBTS4,pathBTS5,pathBTS6,taBTS1,taBTS2,taBTS3,taBTS4,taBTS5,taBTS6
count,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000
mean,1429.730000,-8.070378,-34.894895,116.096927,119.254919,124.059041,116.101179,120.931101,125.168283,0.645000,1.094973,2.485000,0.955000,1.130000,1.105000
std,812.821846,0.004010,0.004296,11.245791,11.771553,8.559135,11.796219,12.480747,13.911055,0.671737,1.393259,0.961456,0.738717,0.840615,0.645023
min,8.000000,-8.077541,-34.905296,87.723333,79.423333,92.690000,87.856667,87.023333,87.523333,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000
25%,685.000000,-8.073634,-34.898285,108.052500,112.481667,119.081667,107.765000,112.215000,113.319167,0.000000,1.000000,2.000000,1.000000,0.000000,1.000000
50%,1503.000000,-8.069830,-34.895534,117.456667,119.456667,124.115000,116.390000,122.390000,129.406667,1.000000,1.000000,2.000000,1.000000,1.000000,1.000000
75%,2008.500000,-8.068513,-34.891111,124.327500,128.577500,129.783750,124.273333,130.231667,136.040000,1.000000,1.000000,3.000000,1.000000,2.000000,2.000000
max,2928.000000,-8.060637,-34.885357,139.390000,143.390000,143.490000,142.656667,144.490000,149.423333,2.000000,11.864499,5.000000,3.000000,3.000000,2.000000


In [149]:
lista = data['pontoId'].tolist()
# dpid = data['pontoId']
data.drop_duplicates(inplace=True)
data.shape
data = data.drop(['pontoId'], axis=1)
data['taBTS2'] = data['taBTS2'].fillna(data['taBTS2'].mean())
data['taBTS4'] = data['taBTS4'].fillna(data['taBTS4'].mean())
data.describe()

,lat,lon,pathBTS1,pathBTS2,pathBTS3,pathBTS4,pathBTS5,pathBTS6,taBTS1,taBTS2,taBTS3,taBTS4,taBTS5,taBTS6
count,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000
mean,-8.070378,-34.894895,116.096927,119.254919,124.059041,116.101179,120.931101,125.168283,0.645000,1.094973,2.485000,0.955000,1.130000,1.105000
std,0.004010,0.004296,11.245791,11.771553,8.559135,11.796219,12.480747,13.911055,0.671737,1.393259,0.961456,0.738717,0.840615,0.645023
min,-8.077541,-34.905296,87.723333,79.423333,92.690000,87.856667,87.023333,87.523333,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000
25%,-8.073634,-34.898285,108.052500,112.481667,119.081667,107.765000,112.215000,113.319167,0.000000,1.000000,2.000000,1.000000,0.000000,1.000000
50%,-8.069830,-34.895534,117.456667,119.456667,124.115000,116.390000,122.390000,129.406667,1.000000,1.000000,2.000000,1.000000,1.000000,1.000000
75%,-8.068513,-34.891111,124.327500,128.577500,129.783750,124.273333,130.231667,136.040000,1.000000,1.000000,3.000000,1.000000,2.000000,2.000000
max,-8.060637,-34.885357,139.390000,143.390000,143.490000,142.656667,144.490000,149.423333,2.000000,11.864499,5.000000,3.000000,3.000000,2.000000


In [150]:
x = data.drop(['lat', 'lon'], axis=1)

transformerX = StandardScaler().fit(x)
x_scaled = transformerX.transform(x)

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:4: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  after removing the cwd from sys.path.


In [151]:
bts1_coord = np.tile(bts_data[['lat', 'lon']].iloc[0].values, (1500,1))
data['distBTS1'] = getDistance(data[['lat', 'lon']].values, bts1_coord) 

bts2_coord = np.tile(bts_data[['lat', 'lon']].iloc[1].values, (1500,1))
data['distBTS2'] = getDistance(data[['lat', 'lon']].values, bts2_coord) 

bts3_coord = np.tile(bts_data[['lat', 'lon']].iloc[2].values, (1500,1))
data['distBTS3'] = getDistance(data[['lat', 'lon']].values, bts3_coord) 

bts4_coord = np.tile(bts_data[['lat', 'lon']].iloc[3].values, (1500,1))
data['distBTS4'] = getDistance(data[['lat', 'lon']].values, bts4_coord) 

bts5_coord = np.tile(bts_data[['lat', 'lon']].iloc[4].values, (1500,1))
data['distBTS5'] = getDistance(data[['lat', 'lon']].values, bts5_coord) 

bts6_coord = np.tile(bts_data[['lat', 'lon']].iloc[5].values, (1500,1))
data['distBTS6'] = getDistance(data[['lat', 'lon']].values, bts6_coord) 

y = data[['distBTS1', 'distBTS2', 'distBTS3', 'distBTS4', 'distBTS5', 'distBTS6']]
y.describe()

,distBTS1,distBTS2,distBTS3,distBTS4,distBTS5,distBTS6
count,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000
mean,0.649203,0.798468,1.644358,0.798468,0.906277,0.848774
std,0.323979,0.397909,0.490595,0.397909,0.440169,0.332859
min,0.079260,0.042575,0.870382,0.042575,0.029781,0.032542
25%,0.391395,0.555898,1.309590,0.555898,0.504826,0.618176
50%,0.590712,0.741243,1.606962,0.741243,0.903438,0.794047
75%,0.882874,0.989815,1.974628,0.989815,1.266405,1.163163
max,1.628075,2.059792,2.795245,2.059792,1.845185,1.522438


In [152]:
transformerY = StandardScaler().fit(y)
y_scaled = transformerY.transform(y)


X_train, X_test, y_train, y_test = train_test_split(x_scaled, y_scaled, test_size = 0.3)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

invX = transformerX.inverse_transform
invY = transformerY.inverse_transform

((140, 12), (60, 12), (140, 6), (60, 6))


In [153]:
knn = MultiOutputRegressor(KNeighborsRegressor(n_neighbors=3))
knn.fit(X_train, y_train)

knnreal = invY(y_test)
knnpredict = invY(knn.predict(X_test))

print (np.mean((knnpredict - knnreal)**2, axis=0))

[ 0.00794506  0.0090182   0.01133815  0.0090182   0.0083815   0.00923049]


/usr/local/lib/python2.7/dist-packages/sklearn/externals/joblib/parallel.py:268: DeprecationWarning: check_pickle is deprecated in joblib 0.12 and will be removed in 0.13
  ' removed in 0.13', DeprecationWarning)
/usr/local/lib/python2.7/dist-packages/sklearn/externals/joblib/parallel.py:268: DeprecationWarning: check_pickle is deprecated in joblib 0.12 and will be removed in 0.13
  ' removed in 0.13', DeprecationWarning)
/usr/local/lib/python2.7/dist-packages/sklearn/externals/joblib/parallel.py:268: DeprecationWarning: check_pickle is deprecated in joblib 0.12 and will be removed in 0.13
  ' removed in 0.13', DeprecationWarning)
/usr/local/lib/python2.7/dist-packages/sklearn/externals/joblib/parallel.py:268: DeprecationWarning: check_pickle is deprecated in joblib 0.12 and will be removed in 0.13
  ' removed in 0.13', DeprecationWarning)
/usr/local/lib/python2.7/dist-packages/sklearn/externals/joblib/parallel.py:268: DeprecationWarning: check_pickle is deprecated in joblib 0.12 and w

In [154]:
d2 = invY(knn.predict(x_scaled))

df2 = pd.DataFrame(d2, columns=['distBTS1', 'distBTS2', 'distBTS3', 'distBTS4', 'distBTS5', 'distBTS6'])

df2.describe()

/usr/local/lib/python2.7/dist-packages/sklearn/externals/joblib/parallel.py:268: DeprecationWarning: check_pickle is deprecated in joblib 0.12 and will be removed in 0.13
  ' removed in 0.13', DeprecationWarning)
/usr/local/lib/python2.7/dist-packages/sklearn/externals/joblib/parallel.py:268: DeprecationWarning: check_pickle is deprecated in joblib 0.12 and will be removed in 0.13
  ' removed in 0.13', DeprecationWarning)
/usr/local/lib/python2.7/dist-packages/sklearn/externals/joblib/parallel.py:268: DeprecationWarning: check_pickle is deprecated in joblib 0.12 and will be removed in 0.13
  ' removed in 0.13', DeprecationWarning)
/usr/local/lib/python2.7/dist-packages/sklearn/externals/joblib/parallel.py:268: DeprecationWarning: check_pickle is deprecated in joblib 0.12 and will be removed in 0.13
  ' removed in 0.13', DeprecationWarning)
/usr/local/lib/python2.7/dist-packages/sklearn/externals/joblib/parallel.py:268: DeprecationWarning: check_pickle is deprecated in joblib 0.12 and w

,distBTS1,distBTS2,distBTS3,distBTS4,distBTS5,distBTS6
count,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000
mean,0.643633,0.792849,1.640606,0.792849,0.899999,0.845054
std,0.309712,0.381081,0.486030,0.381081,0.433685,0.319572
min,0.084152,0.108926,0.871956,0.108926,0.070435,0.120696
25%,0.385140,0.582421,1.306102,0.582421,0.493612,0.628370
50%,0.572840,0.759091,1.567814,0.759091,0.900782,0.800580
75%,0.876659,0.979561,1.969770,0.979561,1.274230,1.021302
max,1.364748,1.844144,2.776572,1.844143,1.666753,1.512019


In [155]:
data['distBTS1'] = df2['distBTS1']
data['distBTS2'] = df2['distBTS2']
data['distBTS3'] = df2['distBTS3']
data['distBTS4'] = df2['distBTS4']
data['distBTS5'] = df2['distBTS5']
data['distBTS6'] = df2['distBTS6']
data.describe()

,lat,lon,pathBTS1,pathBTS2,pathBTS3,pathBTS4,pathBTS5,pathBTS6,taBTS1,taBTS2,taBTS3,taBTS4,taBTS5,taBTS6,distBTS1,distBTS2,distBTS3,distBTS4,distBTS5,distBTS6
count,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000
mean,-8.070378,-34.894895,116.096927,119.254919,124.059041,116.101179,120.931101,125.168283,0.645000,1.094973,2.485000,0.955000,1.130000,1.105000,0.643633,0.792849,1.640606,0.792849,0.899999,0.845054
std,0.004010,0.004296,11.245791,11.771553,8.559135,11.796219,12.480747,13.911055,0.671737,1.393259,0.961456,0.738717,0.840615,0.645023,0.309712,0.381081,0.486030,0.381081,0.433685,0.319572
min,-8.077541,-34.905296,87.723333,79.423333,92.690000,87.856667,87.023333,87.523333,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.084152,0.108926,0.871956,0.108926,0.070435,0.120696
25%,-8.073634,-34.898285,108.052500,112.481667,119.081667,107.765000,112.215000,113.319167,0.000000,1.000000,2.000000,1.000000,0.000000,1.000000,0.385140,0.582421,1.306102,0.582421,0.493612,0.628370
50%,-8.069830,-34.895534,117.456667,119.456667,124.115000,116.390000,122.390000,129.406667,1.000000,1.000000,2.000000,1.000000,1.000000,1.000000,0.572840,0.759091,1.567814,0.759091,0.900782,0.800580
75%,-8.068513,-34.891111,124.327500,128.577500,129.783750,124.273333,130.231667,136.040000,1.000000,1.000000,3.000000,1.000000,2.000000,2.000000,0.876659,0.979561,1.969770,0.979561,1.274230,1.021302
max,-8.060637,-34.885357,139.390000,143.390000,143.490000,142.656667,144.490000,149.423333,2.000000,11.864499,5.000000,3.000000,3.000000,2.000000,1.364748,1.844144,2.776572,1.844143,1.666753,1.512019


In [156]:
x = data.drop(['lat', 'lon'], axis=1)
x.describe()

transformerX = StandardScaler().fit(x)
x_scaled = transformerX.transform(x)

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:5: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  """


In [157]:
y = data[['lat', 'lon']]
y.describe()

,lat,lon
count,200.000000,200.000000
mean,-8.070378,-34.894895
std,0.004010,0.004296
min,-8.077541,-34.905296
25%,-8.073634,-34.898285
50%,-8.069830,-34.895534
75%,-8.068513,-34.891111
max,-8.060637,-34.885357


In [158]:
transformerY = StandardScaler().fit(y)
y_scaled = transformerY.transform(y)


X_train, X_test, y_train, y_test = train_test_split(x_scaled, y_scaled, test_size = 0.3)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

invX = transformerX.inverse_transform
invY = transformerY.inverse_transform

((140, 18), (60, 18), (140, 2), (60, 2))


In [159]:
knn = MultiOutputRegressor(KNeighborsRegressor(n_neighbors=3))
knn.fit(X_train, y_train)

knnreal = invY(y_test)
knnpredict = invY(knn.predict(X_test))

print(getDistance(knnreal, knnpredict).mean())

0.107203507154


/usr/local/lib/python2.7/dist-packages/sklearn/externals/joblib/parallel.py:268: DeprecationWarning: check_pickle is deprecated in joblib 0.12 and will be removed in 0.13
  ' removed in 0.13', DeprecationWarning)
/usr/local/lib/python2.7/dist-packages/sklearn/externals/joblib/parallel.py:268: DeprecationWarning: check_pickle is deprecated in joblib 0.12 and will be removed in 0.13
  ' removed in 0.13', DeprecationWarning)


In [160]:
svr = svm.SVR()
regr = MultiOutputRegressor(svr)
regr.fit(X_train,y_train)

svmreal = invY(y_test)
svmpredict = invY(regr.predict(X_test))

print(getDistance(svmreal, svmpredict).mean())

0.0938539715776


In [161]:
import math
import mpu

L = 3
B = 6

RES_list = []
# RES_list.append(['pontoId', 'lat','lon','lat_pred','lon_pred','erro_loc'])

BTS_mtz = np.array([[1.]*L]*B)

def coord_conv(lat,lon,alt):
	lat = float(lat)
	lon = float(lon)
	earthR = 6371
	x = earthR *(math.cos(math.radians(lat)) * math.cos(math.radians(lon)))
	y = earthR *(math.cos(math.radians(lat)) * math.sin(math.radians(lon)))
	z = earthR *(math.sin(math.radians(lat)))

	return np.array([x,y,z])

def km_dist(p, lat, lon):
	return mpu.haversine_distance((data['lat'][p], data['lon'][p]) , (lat,lon))

def initalize_bts():
    BTS_mtz[0] = coord_conv(bts1_coord[0][0], bts1_coord[0][1],0)
    BTS_mtz[1] = coord_conv(bts2_coord[0][0], bts2_coord[0][1],0)
    BTS_mtz[2] = coord_conv(bts3_coord[0][0], bts3_coord[0][1],0)
    BTS_mtz[3] = coord_conv(bts4_coord[0][0], bts4_coord[0][1],0)
    BTS_mtz[4] = coord_conv(bts5_coord[0][0], bts5_coord[0][1],0)
    BTS_mtz[5] = coord_conv(bts6_coord[0][0], bts6_coord[0][1],0)
    


def tri_lat(p_index,i_index,j_index,k_index, dist_mtz):
	#print p_index, i_index, j_index, k_index
	# convert_np() #initializate BTS_mtz
	# dist_mtz = dist_array() #m


	P1 = np.copy(BTS_mtz[i_index])
	P2 = np.copy(BTS_mtz[j_index])
	P3 = np.copy(BTS_mtz[k_index])

	Dist1 = np.copy(df2['distBTS'+str(i_index+1)][p_index])
	Dist2 = np.copy(df2['distBTS'+str(j_index+1)][p_index])
	Dist3 = np.copy(df2['distBTS'+str(k_index+1)][p_index])

	#Translade circles references (BTS)
	ex = (P2 - P1)/(np.linalg.norm(P2 - P1))
	i = np.dot(ex,P3 - P1)
	ey = (P3 - P1 - i*ex)/(np.linalg.norm(P3 - P1 - i*ex))
	ez = np.cross(ex,ey)
	d = np.linalg.norm(P2 - P1)
	j = np.dot(ey,P3 - P1)

	#plug and chug
	x = (pow(Dist1,2) - pow(Dist2,2) + pow(d,2))/(2*d)
	y = (( pow(Dist1,2) - pow(Dist3,2) + pow(i,2) + pow(j,2))/(2*j))
	- ((i/j) *x)

	#only contendoa
	test = (pow(Dist1,2) - pow(x,2) - pow(y,2))
	if(test < 0):
		return (float("inf"),0,0)
	else:
		z = np.sqrt(test)


	triPt = P1 + x*ex + y*ey + z*ez

	#Back
	earthR = 6371
	if(triPt[2]/earthR < -1 or triPt[2]/earthR > 1):
		# print "Forced again"
		# return float("inf")
		return (float("inf"),0,0)
	lat = math.degrees(math.asin(triPt[2] / earthR))
	lon = math.degrees(math.atan2(triPt[1],triPt[0]))

	dist = km_dist(p_index,lat,lon)
	return (dist,lat,lon)


#Menor erro com determinada BTS
def min_error(p_index, bst_index, dist_mtz):
	list = []
	for i in range (0,6):
		if(i!=bst_index):
			for j in range (0,6):
				if(j!=bst_index and j!= i):
					list.append( tri_lat(p_index,bst_index,i,j,dist_mtz))
					# print "i,j,k -", bst_index,i,j
					#print "C,I,J: ",bst_index, i,j
	#print "Min Dist - ", min(list)*1000, "m"
	return min(list)

#Menor erro com combinacao de todas as BTSs
def best_error(p_index,dist_mtz):
	list = []
	for i in range (0,6):
		list.append(min_error(p_index,i,dist_mtz))

	return min(list)


def all_points(dist_mtz,points):
	sum = 0
	p = points
	for i in range(0,points):
		RES_list.append([lista[i]])    
		(err,lat,lon) = best_error(i,dist_mtz)
		RES_list[i].append(data['lat'][i])
		RES_list[i].append(data['lon'][i])
		RES_list[i].append(lat)
		RES_list[i].append(lon)
		RES_list[i].append(err*1000)
		# print "Point:", i, " Min Error ", err*1000, "m"
		# print "Real: ", LOC_mtz[i][0],",",LOC_mtz[i][1]
		# print "Pred: ", lat,",",lon, "\n"
		if(err != float("inf")):
			sum += err
		else:
			p = p-1
	print ("Average error - ", (sum/p)*1000, "m" , "with:", p ," points" )         

In [162]:
initalize_bts()
print (BTS_mtz)

df2.describe()

# print (df2['distBTS1'][0])

# Dist1 = np.copy(df2['distBTS'+str(1)][1499])

# data.describe()

all_points(df2,200)



[[ 5173.92348894 -3608.40207688  -894.19862566]
 [ 5173.70776525 -3608.50515716  -895.0304411 ]
 [ 5172.85869409 -3609.7100785   -895.07937088]
 [ 5173.70776522 -3608.50515722  -895.0304411 ]
 [ 5174.16013139 -3608.12715558  -893.93868017]
 [ 5173.85462323 -3608.60385761  -893.78271214]]
('Average error - ', 52.95870877893611, 'm', 'with:', 200, ' points')


In [163]:
labels = ['pontoId', 'lat','lon','lat_pred','lon_pred','erro_loc']

dfres = pd.DataFrame(RES_list,columns=labels)

dfres.describe()
dfres.to_csv('Dados/Resultados.csv', index=False)


In [164]:
dfres.describe()



,pontoId,lat,lon,lat_pred,lon_pred,erro_loc
count,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000
mean,1429.730000,-8.070378,-34.894895,-8.070327,-34.894819,52.958709
std,812.821846,0.004010,0.004296,0.003893,0.004076,58.203093
min,8.000000,-8.077541,-34.905296,-8.077436,-34.902992,1.032272
25%,685.000000,-8.073634,-34.898285,-8.073356,-34.897683,15.007461
50%,1503.000000,-8.069830,-34.895534,-8.069792,-34.895434,36.116572
75%,2008.500000,-8.068513,-34.891111,-8.068533,-34.891687,67.475525
max,2928.000000,-8.060637,-34.885357,-8.061430,-34.886101,365.331794
